# Plant growth simulation
This notebook contains a simple simulation of a plants growing path based on sun direction.

## Intro

**Disclaimer**: There be errors. Surely this isn't 100% good and correct. Equations could be wrong, bugs, wrong calculations... But for a simple simulation it's ok.

TODO:
- write Intro
- write how to get equations
- idea: plants grow continuously; use momentum in growth direction

In [1]:
# Install prerequisites
# pip install -r requirements.txt
# conda install -c plotly plotly-orca

In [2]:
from plotly import graph_objects as go
import numpy as np

In [4]:
## plant growth speed
dl = 0.147058824 #water spinach
#dl = 7.62 #bamboo 3ft/day
##TODO find orchid growth speed

def sun_trajectory(t_hours=24, v_platform=0):
    sun = np.array([
        [
            np.cos(2 * np.pi * v_platform * h/24) * np.cos(2*np.pi*h/24) + np.sin(2 * np.pi * v_platform * h/24) * np.cos(np.pi/4) * np.sin(2*np.pi*h/24),
            -np.sin(2 * np.pi * v_platform * h/24) * np.cos(2*np.pi*h/24) + np.cos(2 * np.pi * v_platform * h/24) * np.cos(np.pi/4) * np.sin(2*np.pi*h/24),
            -np.sin(np.pi/4) * np.sin(2*np.pi*h/24)
        ] for h in np.arange(0,t_hours+1,1)
    ])
    
    return sun

In [64]:
sun = sun_trajectory(v_platform=-1)

data = go.Scatter3d(
    x=sun[:,0],
    y=sun[:,1],
    z=sun[:,2],
    marker=dict(
        size=8,
        color=sun[:,2],
        colorscale="bluered"
    ),
    line=dict(
        color=sun[:,2],
        colorscale="bluered"
    )
)

fig = go.Figure(
    data,
    layout=go.Layout(
        scene={
            "aspectmode": "cube",
            "xaxis": {"range": [-1,1]},
            "yaxis": {"range": [-1,1]},
            "zaxis": {"range": [-1,1]}
        }
    )
)
fig.show()

In [62]:
sun_paths = [sun_trajectory(v_platform=v) for v in np.arange(-3,5,1)]

data = [
        go.Scatter3d(
        x=sun[:,0],
        y=sun[:,1],
        z=sun[:,2]
        )
 for sun in sun_paths]

fig = go.Figure(
    data,
    layout=go.Layout(
        scene={
            "aspectmode": "cube",
            "xaxis": {"range": [-1,1]},
            "yaxis": {"range": [-1,1]},
            "zaxis": {"range": [-1,1]}
        }
    )
)
fig.show()

In [8]:
def get_growth_path(v_platform=0, duration=24):
    # start growing from (0,0,0)
    plant = np.zeros((1,3))

    # sun direction
    sun = sun_trajectory(v_platform=v_platform, t_hours=duration)

    for h in range(duration):
        if sun[h,2] > 0:
            plant = np.vstack(( 
                plant,
                plant[-1,:] + dl * sun[h]
            ))
        else:
            plant = np.vstack(( 
                plant,
                plant[-1,:]
            ))

    return plant

In [65]:
plant_path = get_growth_path(duration=24*10, v_platform=0.5)

data = go.Scatter3d(
    x=plant_path[:,0],
    y=plant_path[:,1],
    z=plant_path[:,2],
    marker=dict(
        size=2
    )
)

fig = go.Figure(
    data,
    layout=go.Layout(
        scene={
            "aspectmode": "data"
        }
    )
)
fig.show()

In [67]:
v_min = -10.0
v_max = 10.0
v_step = 0.2

plants = []
speed = []
for v in np.arange(v_min, v_max+v_step, v_step):
    plants.append(get_growth_path(v_platform=v, duration=24*10))
    speed.append(v)

data = []
for v, plant in zip(speed, plants):
    color = int((v-v_min)/(v_max-v_min)/2.0*100)
    data.append(
        go.Scatter3d(
            x=plant[:,0],
            y=plant[:,1],
            z=plant[:,2],
            marker={
                "size": 1,
                "color": f"hsv({color}%,100%,50%)",
            },
            line={
                "color": f"hsv({color}%,100%,50%)",
            },
            name=f"speed={np.round(v,1)}"
        )
    )
fig = go.Figure(
    data,
    layout=go.Layout(
        scene={
            "aspectmode": "data"
        }
    )
)
fig.show()


In [68]:
## high level metrics visualization
speed = np.array(speed)
plants = np.array(plants)

means = plants.mean(axis=1)
stds = plants.std(axis=1)
dists = np.linalg.norm(plants[:,-1,:], axis=1)
dists_xy = np.linalg.norm(plants[:,-1,0:2], axis=1)

data = []
data.append(
    go.Scatter(
        x=speed,
        y=means[:,0],
        name="avg(x)",
        visible="legendonly"
    )
)
data.append(
    go.Scatter(
        x=speed,
        y=means[:,1],
        name="avg(y)",
        visible="legendonly"
    )
)
data.append(
    go.Scatter(
        x=speed,
        y=stds[:,0],
        name="std(x)",
        visible="legendonly"
    )
)
data.append(
    go.Scatter(
        x=speed,
        y=stds[:,1],
        name="std(y)",
        visible="legendonly"
    )
)
data.append(
    go.Scatter(
        x=speed,
        y=np.abs(means[:,0])+np.abs(means[:,1]),
        name="abs(avg(x))+abs(avg(y))"
    )
)
data.append(
    go.Scatter(
        x=speed,
        y=np.abs(stds[:,0])+np.abs(stds[:,1]),
        name="abs(std(x))+abs(std(y))"
    )
)
data.append(
    go.Scatter(
        x=speed,
        y=np.abs(means[:,0]/means[:,0].max()) \
            + np.abs(means[:,1]/means[:,1].max()) \
            + np.abs(stds[:,0]/stds[:,0].max()) \
            + np.abs(stds[:,1]/stds[:,1].max()),
        name="abs(avg(x))+abs(avg(y))+abs(std(x))+abs(std(y))"
    )
)
data.append(
    go.Scatter(
        x=speed,
        y=dists,
        name="distance from center",
        visible="legendonly"
    )
)
data.append(
    go.Scatter(
        x=speed,
        y=dists_xy,
        name="distance from center in plane"
    )
)
fig = go.Figure(
    data,
    layout=go.Layout(
        xaxis=dict(
            title="speed (rot/day)"
        )
    )
)
fig.show()

In [70]:
dists_2d = np.linalg.norm(plants[:,:,0:2], axis=2)
data_3d_dists = [
    go.Scatter(
        # x=,
        y=dists_2d[i,:],
        name=f"speed={np.round(speed[i],1)}"
    ) for i in range(len(speed))
]
fig_2d_dists = go.Figure(data_3d_dists)
fig_2d_dists.show()

In [86]:
dists_2d_cumu = np.cumsum(dists_2d, axis=1)
data_3d_dists = [
    go.Scatter(
        # x=,
        y=dists_2d_cumu[i,:],
        name=f"speed={np.round(speed[i],1)}"
    ) for i in range(len(speed))
]
fig_2d_dists = go.Figure(
    data_3d_dists,
    layout=go.Layout(
        yaxis=dict(
            type="log"
        )
    )
)
fig_2d_dists.show()

15.209765765023093